## 0. Getting setup


In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12 or int(torch.__version__.split(".")[0]) == 2, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

torch version: 2.6.0+cu118
torchvision version: 0.21.0+cu118


In [3]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from PyTorch_Going_Modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
    !git clone https://github.com/CYBki/Factbites
    !move Factbites/PyTorch_Going_Modular .
    !move Factbites/helper_functions.py . # get the helper_functions.py script
    !rmdir /s /q Factbites
    from PyTorch_Going_Modular.going_modular import data_setup, engine
    from helper_functions import download_data, set_seeds, plot_loss_curves

[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.


Cloning into 'Factbites'...
Updating files:  56% (1239/2192)
Updating files:  57% (1250/2192)
Updating files:  58% (1272/2192)
Updating files:  59% (1294/2192)
Updating files:  60% (1316/2192)
Updating files:  61% (1338/2192)
Updating files:  62% (1360/2192)
Updating files:  63% (1381/2192)
Updating files:  64% (1403/2192)
Updating files:  65% (1425/2192)
Updating files:  66% (1447/2192)
Updating files:  67% (1469/2192)
Updating files:  68% (1491/2192)
Updating files:  69% (1513/2192)
Updating files:  70% (1535/2192)
Updating files:  71% (1557/2192)
Updating files:  72% (1579/2192)
Updating files:  73% (1601/2192)
Updating files:  74% (1623/2192)
Updating files:  75% (1644/2192)
Updating files:  76% (1666/2192)
Updating files:  77% (1688/2192)
Updating files:  78% (1710/2192)
Updating files:  79% (1732/2192)
Updating files:  80% (1754/2192)
Updating files:  81% (1776/2192)
Updating files:  82% (1798/2192)
Updating files:  83% (1820/2192)
Updating files:  84% (1842/2192)
Updating files:

        1 dir(s) moved.


The syntax of the command is incorrect.


ModuleNotFoundError: No module named 'helper_functions'

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

## 1. Get Data
